# Hamiltonian Decomposition into Pauli Spin Matrices
We are interested in understanding how to decompose Hamiltonians into Pauli strings (Kronecker products of Pauli Spin Matrices) since qubits require us to work with $2 \times 2$ matrices. Hence, translating a Hamiltonian into a set of Pauli Spin Matrices allows us to perform operations on a system to understand its properties.

## Objectives
1. Understand how to encode Hamiltonians into Pauli-Spin Matrices.

### I. Brief Background

The Pauli spin matrices can be written down as

$$ I = \begin{bmatrix}
1 & 0 \\
0 & 1 
\end{bmatrix} \quad
X = \begin{bmatrix}
0 & 1 \\
1 & 0 
\end{bmatrix} \quad
Y = \begin{bmatrix}
0 & -i \\
i & 0 
\end{bmatrix} \quad
Z = \begin{bmatrix}
1 & 0 \\
0 & -1 
\end{bmatrix}
$$ 

These form a complete basis for any real, symmetric (Hermitian) $2 \times 2$ matrix. We can extend this to higher dimensions by noting that the Kronecker/outer product of $N$ Pauli matrices is $2^N$.

### II. The Algorithm
We require an $N$-order square matrix $H$ as our input.
1. If $H$ is not a power-of-two order, increase the size of $H$ using zero-padding. This is motivated by applications involved in finding the lowest eigenvalue of a system (e.g. VQE algorithm), which will be unaffected by this method. (would be good to investigate/prove this)
2. From the set $\{I,X,Y,Z\}$, generate all permutations with $\log_2N$ factors, while allowing repetitions. Note that $N$ here is the dimension of the possibly enlarged matrix from Step 1. Remove all permutations with odd $Y$ since we only require real matrices. Lastly, we generate the set of Kronecker products from the permutations and store said values in a dictionary.
3. Looping through all unique elements of the Hamiltonian $H$, we write an equation where the LHS are elements of the Hamiltonian, and the RHS is the sum of Kronecker products with unknown coefficients. We can write this as one row in the matrix equation $\textbf{M}\textbf{a}=\textbf{h}$, where $\textbf{a}$ is a vector of all the unknowns, and $\textbf{h}$ is the Hamiltonian reformed as a column vector (unique elements only).
4. Use a linear algebra package, e.g. `numpy`, in order to solve the set of equations for the unknowns $\{a_n\}$. Then, we construct a string representation of the matrix decomposition.

### III. Source Code from Reference [1]

In [4]:
def h2zixy(hamiltonian):
    import itertools
    import numpy as np
    
    eps = 1.e-5
    dim = len(hamiltonian)
    
    #Zero Padding of Hamiltonian Matrix
    NextPowTwo = int(2**np.ceil(np.log(dim)/np.log(2)))
    if NextPowTwo != dim:
        diff = NextPowTwo - dim
        hamiltonian = np.hstack((hamiltonian, np.zeros((dim,diff))))
        dim = NextPowTwo
        hamiltonian = np.vstack((hamiltonian, np.zeros((diff,dim))))
    
    Pauli = {'I' : np.array([[1,0],[0,1]]),
             'X' : np.array([[0,1],[1,0]]),
             'Y' : np.array([[0,-1j],[1j,0]]),
             'Z' : np.array([[1,0],[0,-1]])}
    NumTensorRepetitions = int(np.log(dim)/np.log(2))
    NumTotalTensors = 4**NumTensorRepetitions
    PauliKeyList = []
    KeysToDelete = []
    PauliDict = {}
    
    def PauliDictValues(l):
        #returns a generator object, such that calling again yields the next object
        yield from itertools.product(*([l]*NumTensorRepetitions))
    
    for x in PauliDictValues('IXYZ'):
        PauliKeyList.append(''.join(x))
    
    for y in PauliKeyList:
        PauliDict[y] = 0
        
    for key in PauliDict:
        TempList = []
        PauliTensors = []
        NumYs = key.count('Y')
        TempKey = str(key)
    
        if (NumYs % 2) == 0:
            for string in TempKey:
                TempList.append(string)
            
            for SpinMatrix in TempList:
                PauliTensors.append(Pauli[SpinMatrix])
            
            PauliDict[key] = PauliTensors
            CurrentMatrix = PauliDict[key].copy()
        
            for k in range(1, NumTensorRepetitions):
                TemporaryDict = np.kron(CurrentMatrix[k-1], CurrentMatrix[k])
                CurrentMatrix[k] = TemporaryDict
        
            PauliDict[key] = CurrentMatrix[-1]
        
        else:
            KeysToDelete.append(key)
    
    for val in KeysToDelete:
        PauliDict.pop(val)
    
    VecHamElements = np.zeros(int((dim**2+dim)/2))
    h=0
    for i in range (0,dim):
        for j in range(i,dim):
            arr = []
            VecHamElements[h] = hamiltonian[i,j]
            for key in PauliDict:
                TempVar = PauliDict[key]
                arr.append(TempVar[i,j].real)
                
            if i == 0 and j == 0:
                FinalMat = np.array(arr.copy())
            
            else:
                FinalMat = np.vstack((FinalMat,arr))
            
            h += 1
    
    x = np.linalg.solve(FinalMat,VecHamElements)
    a = []
    var_list = list(PauliDict.keys())
    
    for i in range(len(PauliDict)):
        b = x[i]
        if abs(b)>eps:
            a.append(str(b)+'*'+str(var_list[i])+'\n')
            
    DecomposedHam = ''.join(a)
    return DecomposedHam    

### IV. Example for Quantum Harmonic Oscillator

In [11]:
import numpy as np

#here, we define the lattice indices:
def ell(a,n):
    return (2*a - 1 - n)/2

#discrete fourier transform
def DFT(N):
    F = []
    for j in range(1,N+1):
        F_row = []
        for k in range(1,N+1):
            F_row.append( (1/np.sqrt(N)) * np.exp( (2*np.pi*1j*ell(j,N)*ell(k,N) )/N ) )
        F.append(F_row)
    return np.round(F,10)

#position operator
def X_pos(N):
    X = []
    for j in range(1,N+1):
        X_row = []
        for k in range(1,N+1):
            #if-else statements reflects Kronecker-delta function
            if j==k:
                X_row.append(np.sqrt(2*np.pi/N)*ell(j,N))
            else:
                X_row.append(0)
        X.append(X_row)
    return np.round(X,10)

#momentum operator
def P_pos(N):
    F = DFT(N)
    X = X_pos(N)
    F_dag = np.matrix(F).H
    return np.round(np.matmul(F_dag,np.matmul(X,F)),10)

#QHO hamiltonian
def H_harm(N,basis):
    if basis == "pos":
        X = np.matrix(X_pos(N))
        P = P_pos(N)
        return np.round((np.matrix(X)**2 + np.matrix(P)**2)/2,10)
    
#Using the above operators encoded as functions, we build the Quantum Harmonic Oscillator Hamiltonian
H = np.array(np.real(np.matrix(X_pos(4))**2/2 + np.matrix(P_pos(4))**2/2))
print("The Hamiltonian in standard form is:\n\n",H,"\n\n Decomposition into Sum of Pauli Strings:\n\n",h2zixy(H))

The Hamiltonian in standard form is:

 [[ 2.74889357e+00 -5.55360367e-01 -2.56318424e-17  5.55360367e-01]
 [-5.55360367e-01  1.17809725e+00 -5.55360367e-01  2.56318424e-17]
 [-2.56318424e-17 -5.55360367e-01  1.17809725e+00 -5.55360367e-01]
 [ 5.55360367e-01  2.56318424e-17 -5.55360367e-01  2.74889357e+00]] 

 Decomposition into Sum of Pauli Strings:

 1.9634954088821248*II
-0.5553603674881318*IX
-0.5553603674881318*YY
0.7853981634093545*ZZ



## References:
1. Pesce, R. M. N., & Stevenson, P. D. (2021). H2ZIXY: Pauli spin matrix decomposition of real symmetric matrices. arXiv preprint arXiv:2111.00627.
2. Miceli, R., & McGuigan, M. (2018, August). Quantum computation and visualization of hamiltonians using discrete quantum mechanics and ibm qiskit. In 2018 New York Scientific Data Summit (NYSDS) (pp. 1-6). IEEE.